<a href="https://colab.research.google.com/github/brunelboss/UB/blob/main/ejemplo_redes_neuronales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

In [2]:
def transform(stock):
    stock['diff_Open_Close']=stock.Open-stock.Close
    print('PRETRANSFORM')
    df=pd.DataFrame(stock['Adj Close'])
    df['diff Open Close']=stock.Open-stock.Close
    df['Volume']=stock.Volume
    print('TRANSFORM')
    return df

In [3]:
!pip install pykalman
from pykalman import KalmanFilter
def kalman_filter(df):
  x=df['Adj Close']
# Construct a Kalman filter
  kf = KalmanFilter(transition_matrices = [1],
                  observation_matrices = [1],
                  initial_state_mean = 0,
                  initial_state_covariance = 1,
                  observation_covariance=1,
                  transition_covariance=.05)

# Use the observed values of the price to get a rolling mean
  state_means, _ = kf.filter(x.values)
  df['Adj Close']=state_means.flatten()
  
  df=df.drop(df.index.values[:6])
  return df


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 228 kB 5.1 MB/s 
  Created wheel for pykalman: filename=pykalman-0.9.5-py3-none-any.whl size=48462 sha256=c1bdcefb5c243cd861b04ed47ddc63dbd635b9f5a31be3d0e2447cff017be580
  Stored in directory: /root/.cache/pip/wheels/6a/04/02/2dda6ea59c66d9e685affc8af3a31ad3a5d87b7311689efce6
Successfully built pykalman


In [5]:
stock=pd.read_csv('/content/stock_data.csv')


In [ ]:
stock=transform(stock)
stock=kalman_filter(stock)

In [8]:
stock.head(10)

,Adj Close,diff Open Close,Volume
6,1.535510,0.090000,7942700.0
7,2.287710,-2.169998,6577900.0
8,2.252390,1.410000,6684100.0
9,1.777105,1.673333,7641500.0
10,0.955765,1.936667,8598900.0
11,-0.140360,2.200001,9556300.0
12,-1.187368,-0.490002,7883200.0
13,-3.114805,1.609997,10442600.0
14,-3.566950,-2.879997,11617300.0
15,-3.873033,0.209999,8258200.0


In [9]:
x=stock.values

In [10]:
x.shape

(4011, 3)

## Normalizacion datos

In [11]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler0=StandardScaler()
scaler0.fit(x[:,0].reshape(x[:,0].shape[0],1))
x[:,0]=(scaler0.transform(x[:,0].reshape(x[:,0].shape[0],1))).flatten()


scaler1=StandardScaler()
scaler1.fit(x[:,1].reshape(x[:,1].shape[0],1))
x[:,1]=(scaler1.transform(x[:,1].reshape(x[:,1].shape[0],1))).flatten()


scaler2=StandardScaler()
scaler2.fit(x[:,2].reshape(x[:,2].shape[0],1))
x[:,2]=(scaler2.transform(x[:,2].reshape(x[:,2].shape[0],1))).flatten()

In [12]:
def timeseries_to_supervised( X, timesteps,n_target):

    x = np.zeros( [len(X)-(timesteps+n_target), timesteps, X.shape[1] ] )
    y = np.zeros( [len(X)-(timesteps+n_target), n_target ] )

    for t in range(timesteps):
        x[:,t] = X[t:-(timesteps+n_target)+t,:]
    for i in range(n_target):
        y[:,i] = X[timesteps+i:-(n_target-i),0]
    return x,y

In [13]:
X,y=timeseries_to_supervised(x , 30, 2 )

In [14]:
X[-1,:,0]

array([-0.15247687, -0.14888247, -0.14885592, -0.15168367, -0.15912743,
       -0.16920899, -0.18627073, -0.19497004, -0.20223961, -0.20836538,
       -0.21357612, -0.22232978, -0.22509932, -0.22214676, -0.21951317,
       -0.21617916, -0.21228482, -0.20794221, -0.1949196 , -0.18745287,
       -0.17780699, -0.17266327, -0.1675838 , -0.16255573, -0.15756878,
       -0.15526636, -0.1357163 , -0.10236813, -0.07523108, -0.05692091])

In [15]:
y[-1,:]

array([-0.04567225, -0.0400728 ])

## Variabile a predicir

In [16]:
precio_ultimo_dia=X[:,-1,0]
precio_dos_dias_despues=y[:,1]
target=precio_dos_dias_despues-precio_ultimo_dia
target[target<=0]=0
target[target>0]=1
target[:10]

array([1., 1., 1., 1., 1., 1., 0., 0., 0., 0.])

## Variabile INPUT

In [17]:
X.shape

(3979, 30, 3)

## Training y Test

In [18]:
from sklearn.utils import shuffle
X,target=shuffle(X,target)

In [19]:
tt=int(0.9*len(X))


x_train,x_test = X[:tt],X[tt:]
y_train,y_test = target[:tt],target[tt:]


In [20]:
x_train.shape

(3581, 30, 3)

In [21]:
#x_train[1,range(0,90,3)[0]]

In [22]:
y_train.shape

(3581,)

In [23]:
x_test.shape

(398, 30, 3)

In [24]:
y_test.shape

(398,)

In [25]:
y_test=y_test.reshape(y_test.shape[0],1)
y_train=y_train.reshape(y_train.shape[0],1)

In [26]:
y_test.shape

(398, 1)

In [27]:
y_train.shape

(3581, 1)

## Modelo

In [35]:
import keras

from keras.models import Sequential, Model, load_model
from keras.layers import Input, Reshape, Flatten, Activation
from keras.layers import Dropout, BatchNormalization, GaussianNoise
from keras.layers import Dense, LSTM
from keras.constraints import maxnorm
from tensorflow.keras.optimizers import RMSprop, SGD, Adadelta, Adagrad, Adam

In [36]:
_tensor_ = input_tensor = Input( shape=( x_train.shape[1], x_train.shape[2] ), name='main_input' )
_tensor_ = GaussianNoise(0.15)(_tensor_)
_tensor_ = LSTM( 51, activation='relu', kernel_constraint=maxnorm(3.0), return_sequences=True  )(_tensor_)

_tensor_ = Flatten()(_tensor_)

_tensor_ = Dense( 80, activation='relu', kernel_constraint=maxnorm(3.0) )(_tensor_)

_tensor_ = Dense( 50, activation='relu', kernel_constraint=maxnorm(3.0) )(_tensor_)

_tensor_ = Dense( 20, activation='relu', kernel_constraint=maxnorm(3.0) )(_tensor_)

output_tensor = Dense( 1, activation='sigmoid', kernel_constraint=maxnorm(3.0) )(_tensor_)


model=Model( input_tensor, output_tensor )
model.compile( loss='mse', optimizer=Adagrad() ) 
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 main_input (InputLayer)     [(None, 30, 3)]           0         
                                                                 
 gaussian_noise (GaussianNoi  (None, 30, 3)            0         
 se)                                                             
                                                                 
 lstm (LSTM)                 (None, 30, 51)            11220     
                                                                 
 flatten (Flatten)           (None, 1530)              0         
                                                                 
 dense (Dense)               (None, 80)                122480    
                                                                 
 dense_1 (Dense)             (None, 50)                4050      
                                                             

In [ ]:
model.fit( x_train, y_train, batch_size=300, epochs=150, shuffle=True, verbose=1 )

In [38]:
y_pred = model.predict( x_test )


In [39]:
y_pred_class=np.copy(y_pred)
y_pred_class[y_pred<0.6]=0
y_pred_class[y_pred>=0.6]=1

In [40]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test,y_pred_class)

array([[161,   0],
       [237,   0]])

In [41]:
(y_test==y_pred_class).sum()

161

In [42]:
y_test.shape

(398, 1)